# Load Dependencies

In [2]:
!pip install nltk
!pip install transformers
!pip install collections
!pip install rouge_score
!pip install pycocoevalcap
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=dae762ae5fc1be9909cea0f0fcc2d6de3bef7180b88ac598154b6d641451f6cc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_scor

# Load Finetuned Model from Hugging Face

In [3]:
from transformers import AutoProcessor, Pix2StructForConditionalGeneration

processor = AutoProcessor.from_pretrained("oroikon/ft_pix2struct_chart_captioning")
model = Pix2StructForConditionalGeneration.from_pretrained("oroikon/ft_pix2struct_chart_captioning")

# Load Evaluation Dataset from Hugging Face

In [4]:
from datasets import load_dataset
dataset = load_dataset('hk-kaden-kim/pix2struct-chartcaptioning')
eval_datasets = dataset['test']



Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1373 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/199 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/399 [00:00<?, ? examples/s]

# Dataset Preparation

In [6]:
# Function to filter datasets based on conditions
def filter_dataset(dataset, chart_type, chart_element):
    return dataset.filter(lambda example: example['chartType'] == chart_type and example['chartElement'] == chart_element)


In [8]:
from itertools import product

chart_types = ['line', 'bar']
chart_elements = ['SubPlot', 'FullCover', 'NoTitle', 'NoGrid', 'OnePlot', 'NoAxisLabel']

# Create all the possible combinations of'chartType' and 'chartElement' to achieve several plot configurations scenarios
combinations = list(product(chart_types, chart_elements))

In [9]:
# Dictionary to store filtered datasets
eval_datasets_dict = {}

for combo in combinations:
    # Datasets names based on plot configurations
    dataset_name = f"{combo[0]}_{combo[1]}".lower()

    # Filter the datasets
    eval_dataset_dict = filter_dataset(eval_datasets, combo[0], combo[1])

    # Creation of the dictionary
    eval_datasets_dict[dataset_name] = eval_dataset_dict


Filter:   0%|          | 0/399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/399 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/399 [00:00<?, ? examples/s]

In [11]:
eval_datasets_dict

{'line_subplot': Dataset({
     features: ['image', 'caption', 'chartType', 'chartElement', 'dataType'],
     num_rows: 30
 }),
 'line_fullcover': Dataset({
     features: ['image', 'caption', 'chartType', 'chartElement', 'dataType'],
     num_rows: 11
 }),
 'line_notitle': Dataset({
     features: ['image', 'caption', 'chartType', 'chartElement', 'dataType'],
     num_rows: 16
 }),
 'line_nogrid': Dataset({
     features: ['image', 'caption', 'chartType', 'chartElement', 'dataType'],
     num_rows: 7
 }),
 'line_oneplot': Dataset({
     features: ['image', 'caption', 'chartType', 'chartElement', 'dataType'],
     num_rows: 35
 }),
 'line_noaxislabel': Dataset({
     features: ['image', 'caption', 'chartType', 'chartElement', 'dataType'],
     num_rows: 21
 }),
 'bar_subplot': Dataset({
     features: ['image', 'caption', 'chartType', 'chartElement', 'dataType'],
     num_rows: 72
 }),
 'bar_fullcover': Dataset({
     features: ['image', 'caption', 'chartType', 'chartElement', 'dataTyp

# Evaluation on Test Data from each Plot Configuration

In [12]:
import torch
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from pycocoevalcap.meteor.meteor import Meteor
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from PIL import Image
import matplotlib.pyplot as plt



from datasets import load_dataset
import csv



# Define  smoothing function for BLEU Score
chencherry = SmoothingFunction()

meteor_scorer = Meteor()
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Store the average scores
average_scores = {}

for dataset_name, dataset in eval_datasets_dict.items():
  print(f"Processing dataset: {dataset_name}")


  total_rouge_L = 0
  total_rouge_1 = 0
  total_meteor = 0
  total_cider = 0
  num_samples = 0
  bleu = 0

  for idx, example in enumerate(dataset):

      image = example["image"]
      reference = example["caption"]

      # Image preprocessing and caption generation

      input_ids = processor(images=image, return_tensors="pt", max_patches=2048).to(device)
      flattened_patches = input_ids.flattened_patches
      attention_mask = input_ids.attention_mask
      generated_ids = model.generate(flattened_patches=flattened_patches, attention_mask=attention_mask, max_length=100)
      generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

      reference_tokens = word_tokenize(reference)
      generated_tokens = word_tokenize(generated_caption)

      num_samples += 1

      # BLEU score
      score = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=chencherry.method7)
      bleu += score

      # ROUGE score
      scores = rouge_scorer.score(reference, generated_caption)
      total_rouge_L += scores['rougeL'].fmeasure
      total_rouge_1 += scores['rouge1'].fmeasure

      # METEOR score
      image_id = str(idx)
      gts = {image_id : [generated_caption]}
      res = {image_id : [reference]}
      score,scores = meteor_scorer.compute_score(gts, res)
      total_meteor += score


  # Calculate the average scores
  average_rouge_L = total_rouge_L / num_samples
  average_rouge_1 = total_rouge_1 / num_samples
  average_meteor = total_meteor / num_samples
  average_cider = total_cider / num_samples
  avg_bleu = bleu / num_samples


  average_scores[dataset_name] = {
      "ROUGE-L-F1": average_rouge_L,
      "ROUGE-1-F1": average_rouge_1,
      "METEOR": average_meteor,
      "BLEU": avg_bleu,
  }

  print(f"Completed processing dataset: {dataset_name}")

# Save the average scores to a CSV file
with open('average_scores_cases.csv', 'w', newline='') as csvfile:
    fieldnames = ['Dataset', 'ROUGE-L-F1', 'ROUGE-1-F1', 'METEOR', 'BLEU']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for dataset_name, scores in average_scores.items():
        row = {'Dataset': dataset_name}
        row.update(scores)
        writer.writerow(row)

print("All datasets have been processed and scores have been saved.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Processing dataset: line_subplot
Completed processing dataset: line_subplot
Processing dataset: line_fullcover
Completed processing dataset: line_fullcover
Processing dataset: line_notitle
Completed processing dataset: line_notitle
Processing dataset: line_nogrid
Completed processing dataset: line_nogrid
Processing dataset: line_oneplot
Completed processing dataset: line_oneplot
Processing dataset: line_noaxislabel
Completed processing dataset: line_noaxislabel
Processing dataset: bar_subplot
Completed processing dataset: bar_subplot
Processing dataset: bar_fullcover
Completed processing dataset: bar_fullcover
Processing dataset: bar_notitle
Completed processing dataset: bar_notitle
Processing dataset: bar_nogrid
Completed processing dataset: bar_nogrid
Processing dataset: bar_oneplot
Completed processing dataset: bar_oneplot
Processing dataset: bar_noaxislabel
Completed processing dataset: bar_noaxislabel
All datasets have been processed and scores have been saved.


# Store the Total Average Performance and Generated Captions

In [13]:
import torch
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from pycocoevalcap.meteor.meteor import Meteor
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from PIL import Image
import matplotlib.pyplot as plt
from datasets import load_dataset
import csv



# Define  smoothing function for BLEU Score
chencherry = SmoothingFunction()

meteor_scorer = Meteor()
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Store the average scores
average_scores = {}

# Store the generated captions
all_captions = []

total_rouge_L = 0
total_rouge_1 = 0
total_meteor = 0
total_cider = 0
num_samples = 0
bleu = 0

for idx, example in enumerate(eval_datasets):

    image = example["image"]
    reference = example["caption"]

    # Image preprocessing and caption generation

    input_ids = processor(images=image, return_tensors="pt", max_patches=2048).to(device)
    flattened_patches = input_ids.flattened_patches
    attention_mask = input_ids.attention_mask
    generated_ids = model.generate(flattened_patches=flattened_patches, attention_mask=attention_mask, max_length=100)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    all_captions.append(generated_caption)

    reference_tokens = word_tokenize(reference)
    generated_tokens = word_tokenize(generated_caption)

    num_samples += 1

    # BLEU score
    score = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=chencherry.method7)
    bleu += score

    # ROUGE score
    scores = rouge_scorer.score(reference, generated_caption)
    total_rouge_L += scores['rougeL'].fmeasure
    total_rouge_1 += scores['rouge1'].fmeasure

    # METEOR score
    image_id = str(idx)
    gts = {image_id : [generated_caption]}
    res = {image_id : [reference]}
    score,scores = meteor_scorer.compute_score(gts, res)
    total_meteor += score


# Calculate the average scores
average_rouge_L = total_rouge_L / num_samples
average_rouge_1 = total_rouge_1 / num_samples
average_meteor = total_meteor / num_samples
average_cider = total_cider / num_samples
avg_bleu = bleu / num_samples


average_scores[dataset_name] = {
    "ROUGE-L-F1": average_rouge_L,
    "ROUGE-1-F1": average_rouge_1,
    "METEOR": average_meteor,
    "BLEU": avg_bleu,
}

# Writing to csv file
with open('average_scores.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    fieldnames = list(average_scores.keys())
    csv_writer.writerow(fieldnames)
    data_row = list(average_scores.values())
    csv_writer.writerow(data_row)

# Save the generated captions to a CSV file
with open('generated_captions.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['gen_caption']
    writer = csv.writer(csvfile)
    writer.writerow(fieldnames)
    for caption_info in all_captions:
      writer.writerow([caption_info])

print("Scores have been saved")

Scores have been saved to average_scores.csv
